In [1]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt

In [2]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [3]:
A_3_1 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0]
])

A_3_2 = np.array([
    [1, 2, 0, 0],
    [0, 1, -2, 0]
])

A_3_3 = np.array([
    [1, 3, 3, 0],
    [0, -1, 3, -3]
])

In [89]:
k1, k2 = 2, 1

delta = 10

gamma = np.array([
    0, 0, 0, 0#0, 4, 2, -5
])

lam = k1 / k2
y_true = np.array([
    1,
    lam,
    lam**2 + lam,
    lam**3 + 3*(lam**2) + lam
])

y_true += gamma

y_lb = y_true - (delta / 2)
y_ub = y_true + (delta / 2)

y_lb[0] = 1
y_ub[0] = 1

In [90]:
def M_0_constructor(y):
    M = np.array([
        [y[0], y[1]],
        [y[1], y[2]]
    ])
    return M

def M_1_constructor(y):
    M = np.array([
        [y[1], y[2]],
        [y[2], y[3]]
    ])
    return M

In [102]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # model settings
        model.Params.TimeLimit = time_limit
        K = 100

        # variables
        y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
        k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
        M_0 = gp.MVar.fromlist([
            [y[0].item(), y[1].item()],
            [y[1].item(), y[2].item()]
        ])
        M_1 = gp.MVar.fromlist([
            [y[1].item(), y[2].item()],
            [y[2].item(), y[3].item()]
        ])

        # constraints

        # moment bounds
        model.addConstr(y <= y_ub, name="y_UB")
        model.addConstr(y >= y_lb, name="y_LB")

        # moment equations
        model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
        #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
        #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

        # base
        model.addConstr(y[0] == 1, name="y0_base")

        # fixed parameter
        model.addConstr(k[1] == k2, name="k2_fixed")

        # optimize
        solution = {}
        model.setObjective(0, GRB.MINIMIZE)
        model.optimize()
        status = status_codes[model.status]
        print(f"Status {status}")

        # found feasible point
        while status == "OPTIMAL":

            # get moment matrix values
            M_0_val = M_0.X
            M_1_val = M_1.X

            # check semidefinite
            evals_0, evecs_0 = np.linalg.eigh(M_0_val)
            evals_1, evecs_1 = np.linalg.eigh(M_1_val)

            print(evals_0)
            print(evals_1)

            # positive eigenvalues
            if (evals_0 >= 0).all() and (evals_1 >= 0).all():
                print("SDP feasible")
                break

            # negative eigenvalue
            else:

                if evals_0[0] < 0:

                    # get eigenvector
                    v = evecs_0[0]

                    # add cutting plane
                    model.addConstr(v.T @ M_0 @ v >= 0, name="Cut_0")

                    print("M_0 cut")

                if evals_1[0] < 0:

                    # get eigenvector
                    v = evecs_1[0]

                    # add cutting plane
                    model.addConstr(v.T @ M_1 @ v >= 0, name="Cut_1")

                    print("M_1 cut")

                # resolve
                solution = {}
                model.setObjective(0, GRB.MINIMIZE)
                model.optimize()
                status = status_codes[model.status]
                print(f"Status {status}")

Status OPTIMAL
[-2.02282456  4.02282456]
[ 2.95164187 17.0711827 ]
M_0 cut
Status OPTIMAL
[ 0.36852324 11.63147676]
[-3.35379048 22.94483863]
M_1 cut
Status OPTIMAL
[0.09381891 7.43980674]
[ 0.78717896 28.62852613]
SDP feasible


## Notes

Moment equations very good for such a simple model, strong enough that valid moments are feasible and also satisify SDP (so end with no cuts needed) or invalid moments infeasible (so end with no cuts)

Reducing to 1 moment equation weakens constraints so we can test cuts:

Valid moments with very weak bounds (e.g. delta = 10), feasible relaxation point infeasible under SDP so add cut, repeat a few times until feasible (done)

Invalid moments (e.g. gamma = [0, 4, 2, -5]), feasible relaxation point infeasible under SDP so add cut, relaxation is then infeasible (done)

Overall very good performance

### TODO

Clean up code, use functions (model creation, adding cuts, etc) to make loop easier to work with

## Function version

In [111]:
def base_model(model):

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # variables
    y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
    M_0 = gp.MVar.fromlist([
        [y[0].item(), y[1].item()],
        [y[1].item(), y[2].item()]
    ])
    M_1 = gp.MVar.fromlist([
        [y[1].item(), y[2].item()],
        [y[2].item(), y[3].item()]
    ])

    # constraints

    # moment bounds
    model.addConstr(y <= y_ub, name="y_UB")
    model.addConstr(y >= y_lb, name="y_LB")

    # moment equations
    model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
    #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
    #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[1] == k2, name="k2_fixed")

    # variable dict
    variables = {
        'y': y,
        'k': k,
        'M_0': M_0,
        'M_1': M_1
    }

    return model, variables

In [112]:
def optimize(model):

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [122]:
def semidefinite_cut(model, variables, print_evals=False):
    '''
    Check semidefinite feasibility of optimal moment values
    
    Feasible: stop
    Infeasible: add cutting plane

    Returns:
        model
        (bool) semidefinite feasibility
    '''

    # get moment matrix values
    M_0_val = variables['M_0'].X
    M_1_val = variables['M_1'].X

    # check semidefinite
    evals_0, evecs_0 = np.linalg.eigh(M_0_val)
    evals_1, evecs_1 = np.linalg.eigh(M_1_val)

    if print_evals:
        print("Moment matices eigenvalues:")
        print(evals_0)
        print(evals_1)

    # positive eigenvalues
    if (evals_0 >= 0).all() and (evals_1 >= 0).all():

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible")

        if evals_0[0] < 0:

            # get eigenvector
            v = evecs_0[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_0'] @ v >= 0, name="Cut_0")

            print("M_0 cut added\n")

        if evals_1[0] < 0:

            # get eigenvector
            v = evecs_1[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_1'] @ v >= 0, name="Cut_1")

            print("M_1 cut added\n")

    return model, False

In [123]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # construct base model (no semidefinite constraints)
        model, variables = base_model(model)

        # check feasibility
        model, status = optimize(model)

        # while feasible
        while status == "OPTIMAL":

            print("NLP feasible")

            # check semidefinite feasibility
            model, semidefinite_feas = semidefinite_cut(model, variables, print_evals=True)

            # semidefinite feasible
            if semidefinite_feas:
                break

            # semidefinite infeasible
            else:

                # check feasibility
                model, status = optimize(model)

        # if infeasible
        if status == "INFEASIBLE":
            print("SDP infeasible")


NLP feasible
Moment matices eigenvalues:
[-2.02282456  4.02282456]
[ 2.95164187 17.0711827 ]
SDP infeasible
M_0 cut added

NLP feasible
Moment matices eigenvalues:
[ 0.36852324 11.63147676]
[-3.35379048 22.94483863]
SDP infeasible
M_1 cut added

NLP feasible
Moment matices eigenvalues:
[0.09381891 7.43980674]
[ 0.78717896 28.62852613]
SDP feasible

